In [1]:
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.utils import upload_to_neuroglancer as upload
from brainlit.utils import upload_skeleton
import numpy as np
import napari
%gui qt

Using TensorFlow backend.


In [2]:
data_dir = "../../../tests/data_octree"
dest_dir = "./test_precomputed"

num_res = 2

# Uploading Brain Images in the Octree Format
## This is a script for uploading entire brain volumes, or uploading specific resolutions onto AWS or a local directory. 
## Data must be tif files arranged in folders where the highest level corresponds to a single, low res image

### Files should be arranged as octree with 1-8 indicating volume octant, Binary paths are used to stitch together images according to resolution

In [3]:
files, bin_paths, vox_size, tiff_dims = upload.get_volume_info(data_dir, num_res, channel = 0)
print("Low res files: " + str(files[0]))
print("\nHigh res files: " + str(files[1]))
print("---")
print("Single image binary: " + str(bin_paths[0]))
print("\nMultiple image binaries: " + str(bin_paths[1]))

got files and binary representations of paths.
got dimensions of volume
Low res files: []

High res files: [['..', '..', '..', 'tests', 'data_octree', 'default.0.tif']]
---
Single image binary: []

Multiple image binaries: [[]]


### Cloudvolume image layers are created with the number of resolutions in the original data

In [4]:
vols = upload.create_image_layer("file://" + dest_dir,tiff_dims, vox_size, num_res)
print("Number of volumes: " + str(len(vols)))
print("mips: " + str(vols[0].mip) + ' and ' + str(vols[1].mip))
print("Volumes info: " + str(vols[0].info))
print("---")
print("High res volume info: " + str(vols[0].info['scales'][0]))
print("\nLow res volume info: " + str(vols[1].info['scales'][1]))

Number of volumes: 2
mips: 1 and 0
Volumes info: {'data_type': 'uint16', 'num_channels': 1, 'scales': [{'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '6173_6173_6173', 'resolution': [6173, 6173, 6173], 'size': [1056, 800, 416], 'voxel_offset': [0, 0, 0]}, {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '12346_12346_12346', 'resolution': [12346, 12346, 12346], 'size': [528, 400, 208], 'voxel_offset': [0, 0, 0]}], 'type': 'image'}
---
High res volume info: {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '6173_6173_6173', 'resolution': [6173, 6173, 6173], 'size': [1056, 800, 416], 'voxel_offset': [0, 0, 0]}

Low res volume info: {'chunk_sizes': [[66, 50, 52]], 'encoding': 'raw', 'key': '12346_12346_12346', 'resolution': [12346, 12346, 12346], 'size': [528, 400, 208], 'voxel_offset': [0, 0, 0]}


### Uploading can be done with either Joblib parallel or non-parrallel sequential if the cpu power isn't there

In [5]:
%%capture
u1=upload.upload_chunks(vols[0], files[0], bin_paths[0], parallel=False) # Low res
u2=upload.upload_chunks(vols[1], files[1], bin_paths[1], parallel=False) # High res

# Visualize your data with NeuroglancerSession

In [6]:

ngl_sess = NeuroglancerSession(mip = 1, url = "file://" + dest_dir)
from cloudvolume import Bbox
img = ngl_sess.pull_bounds_img(Bbox((0,0,0), (200,200,200)))

Downloading:   0%|          | 0/8 [00:00<?, ?it/s]


In [7]:
with napari.gui_qt():
    ngl_sess.napari_viewer(img)

In [5]:
# Uploading Neuron traces in .swc format locally
swc_dir = "../../../tests/2018-08-01_G-002_consensus.swc"
dest_dir_skel = "./test_precomputed/"

In [6]:
skel = upload_skeleton.swc2skeleton(swc_dir)
vol = upload_skeleton.create_skeleton_layer(
    "file://"+dest_dir_skel, vox_size, tiff_dims, num_res
)
vol.skeleton.upload(skel)

Uploading: 100%|██████████| 1/1 [00:00<00:00, 122.01it/s]


In [7]:
ngl_sess = NeuroglancerSession(mip = 0, url = "file://" + dest_dir)

In [8]:
# ngl_sess.pull_voxel(2, 6, nx=10, ny=10, nz=10) # currently mip mismatch and scale mismatch

In [12]:
from cloudvolume import Bbox
def get_local_volume_around_vertex(ngl_sess, SEGID=2, VID=6, radius=10):
    skel = ngl_sess.cv.skeleton.get(SEGID)
    vertex = skel.vertices[VID]
    scales = np.multiply(ngl_sess.cv.scales[1]["resolution"],2**4) # incomplete data example
    voxel = np.round(np.divide(vertex, scales)).astype(int)
    bounds = Bbox(voxel, voxel)
    seed = bounds.to_list()
    shape = [radius, radius, radius]
    bounds = Bbox(np.subtract(seed[:3], shape), np.add(np.add(seed[3:], shape), 1))
    img = ngl_sess.cv.download(bounds)
    return img

In [13]:
img = get_local_volume_around_vertex(ngl_sess, VID=100)

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


In [14]:
with napari.gui_qt():
    ngl_sess.napari_viewer(img)